In [35]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy

In [53]:
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False
    

In [54]:
TRACK_WORDS = ['Facebook']
TABLE_NAME = "Facebook"
TABLE_ATTRIBUTES = "id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255), \
            polarity INT, subjectivity INT, user_created_at VARCHAR(255), user_location VARCHAR(255), \
            user_description VARCHAR(255), user_followers_count INT, longitude DOUBLE, latitude DOUBLE, \
            retweet_count INT, favorite_count INT"

In [55]:
# Go to http://apps.twitter.com and create an app.
# The consumer key and secret will be generated for you
API_KEY = "vPCOQFvS2vLPohv9Toi4Mbr0W"
API_SECRET_KEY = "x00HrDFfpcX9wCL4DppEubGOU1cJD5QqdOwnjCyuAMbAM3h1nY"

# After the step above, you will be redirected to your app's page.
# Create an access token under the the "Your access token" section
ACCESS_TOKEN = "1325521853901905924-gMaJd2iZKcUQNAFpwe3iYbcUiZ6ooZ"
ACCESS_TOKEN_SECRET = "xG3Yc5GIQm7NyCl5yWWUeAVNQcqkWMQLVR9GWyff3RYN3"

In [56]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [57]:
mydb = mysql.connector.connect(
    host="34.71.119.123",
    user="root",
    passwd="eheye",
    database="project",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    print('inside')
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(TABLE_NAME, TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

inside


In [58]:
auth  = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @SidneyPowell1: And BIG TECH &amp; #Facebook #Google #Twitter are all into suppressing our freedom of speech to challenge this outrageous #E…
Long: None, Lati: None
RT @SB19Official: [Facebook Live NOTICE] 
Our scheduled Facebook Live tonight at 6pm will be delayed at 6:30pm. We are very sorry for letti…
Long: None, Lati: None
RT @PhillDKline: Systematic failures by state and local election officials to adhere to the procedures outlined in Georgia’s election code…
Long: None, Lati: None
RT @LennaLeprena: Tassie stuff...
https://t.co/IO6eWwT8JH https://t.co/jE5LvfYmwP
Long: None, Lati: None
RT @moreland_abby: ❤️if you’re feeling the Christmas spirit &amp; wanna help me by retweeting, i would appreciate it! i just started my own pho…
Long: None, Lati: None
RT @Madnessauto69: This is exactly why the punishment for treason is so severe! We have the evidence to prosecute! The magic ballots are th…
Long: None, Lati: None
RT @espilysbb: (trans) Phi Bell, a creative working w/ Phi Toon's (B

When the Thanksgiving meal is finally served! https://t.co/q2JtIxj1a3
Long: None, Lati: None
Somehow Facebook knows that I both like sequins and 1960s style pumps and they’re milking the HELL out of it more t… https://t.co/yEJb1WLesd
Long: None, Lati: None
😂😂😂Thank you for the new trailer @GemsOfBollywood

https://t.co/rnIqKgcxdO
Long: None, Lati: None
RT @BTS_graphs: Please share this carrd of LGO playlists on BTS Facebook groups, Instagram stories, amino, etc. 

💜 Let’s encourage other A…
Long: None, Lati: None
RT @dangzb: Perfect choice indeed 👍

@mainedcm ☆ #MaineMendoza
#MaineAtBFWBukasNa
Long: None, Lati: None
Yep, it's that time of year again - time for #ScottyTheCoward to piss off on holidays!
https://t.co/nvnlnxwcwo
Long: None, Lati: None
RT @scubatsa31: SHORTAGE OF TOILET PAPER▪︎▪︎▪︎
https://t.co/9MCSffCtjT
Long: None, Lati: None
@Tony__Heller @Twitter @YouTube @Facebook https://t.co/IzQem3Pj60
The best free speech platform that is funded by the users.
Long: None, Lati: None


RT @mrddmia: Translation:

Dear @Facebook &amp; @Twitter:

Thanks for your ongoing censorship of conservatives &amp; other election interference.…
Long: None, Lati: None
RT @Varun_dvn: Arrival time note kar lijiye trailer ka! 28 Nov, 12 PM, Amazon Prime Video ke Facebook aur YouTube page par. Milte hain 🙏 #C…
Long: None, Lati: None
RT @Varun_dvn: Arrival time note kar lijiye trailer ka! 28 Nov, 12 PM, Amazon Prime Video ke Facebook aur YouTube page par. Milte hain 🙏 #C…
Long: None, Lati: None
This isn't amateur hour! I'm still crushing some Call of Duty: Modern Warfare.
https://t.co/SnnqVEGGGh
Long: None, Lati: None
@gabriel_zucman after seeing it advertised on Facebook? I never have. Ever. Do you know anyone who has? I am sure h… https://t.co/6eHPHkyO1z
Long: None, Lati: None
RT @BlackdotTwiggie: Is a date dayyyyy y'all 🗣🗣get ready to chill with Prince  and Rio on Facebook live #RioXPrinceOfGuinness https://t.co/…
Long: None, Lati: None
Please pledge and RT, foster or adopt this beaut